In [8]:
import ray 
import gym 
import numpy as np 
from gym.spaces import Box, Discrete
from ray.rllib.env import MultiAgentEnv
from ray.rllib.policy.policy import PolicySpec

class CustomEnv(MultiAgentEnv, gym.Env):
    def __init__(self, config):
        pass
        # do not require action spaces and observation spaces
    
    def _obs(self):
        return {
            "state": np.random.random(size=(2,3)),
            "otehrs": np.random.random(size=(2,3))
            }

    def step(self, action_dict):
        # done and reward must be dictionary for each agent 
        # observation doesn't have to be agent dictionary -> will be handled at observation function. 
        
        done = {a:np.random.randint(2) for a in action_dict.keys()}
        done["__all__"] = True if (True in done.values()) else False     
           
        return self._obs(), {a:0 for a in action_dict.keys()}, done, {}
    
    def reset(self):
        return self._obs()


In [20]:
from ray.rllib.agents.dqn import DQNTrainer, DQNTorchPolicy
from ray.rllib.agents.ppo import PPOTorchPolicy, PPOTrainer

def observation_function(agent_obs, **kwargs):
    # agents are defined here..?!
    new_obs = {
        "predator_1":np.random.random(6),
        "predator_2":np.random.random(6),
        "prey_1":np.random.random(6),
        "prey_2":np.random.random(6),
        "prey_3":np.random.random(6),
    }
    return new_obs
    

config = {
    "env":CustomEnv,
    "env_config":{},
    "num_workers":0,
    "num_gpus":0,
    "multiagent":
        {"policies":{
            "predator": PolicySpec(
                        policy_class=PPOTorchPolicy,
                        observation_space=Box(-np.inf, np.inf, shape=(6,)),
                        action_space=Discrete(3),
                        config={}),
            "prey": PolicySpec(
                        policy_class=PPOTorchPolicy,
                        observation_space=Box(-np.inf, np.inf, shape=(6,)),
                        action_space=Discrete(3),
                        config={}),
            },
         "policy_mapping_fn": lambda agaent_name : agaent_name.split("_")[0],
         "policies_to_train": ['predator', 'prey'],
         "observation_fn":observation_function
         },
    "framework":"torch"
}

ray.tune.run(
    "PPO", 
    config=config,
    stop={"training_iteration":10}
)

Trial name,status,loc
PPO_CustomEnv_cf5ce_00000,PENDING,


(PPOTrainer pid=18720) 2022-02-24 13:25:36,959	INFO ppo.py:250 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(PPOTrainer pid=18720) 2022-02-24 13:25:36,959	INFO trainer.py:792 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(PPOTrainer pid=18720) 2022-02-24 13:25:36,989	WARNING deprecation.py:46 -- DeprecationWarning: `rllib.env.remote_vector_env.RemoteVectorEnv` has been deprecated. Use `ray.rllib.env.remote_base_env.RemoteBaseEnv` instead. This will raise an error in the future!
(PPOTrainer pid=18720) 2022-02-24 13:25:36,990	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(PPOTrainer pid=18720) 2022-02-24 13:25:36,995	WARNING deprecation.py:46 -- DeprecationWarning: `policy_mapping_fn(agent_id)` has been deprecated. Use `policy_mapping_fn(agent_id, episode, worker, **kwargs)` instead. This

Trial name,status,loc
PPO_CustomEnv_cf5ce_00000,RUNNING,127.0.0.1:18720


Trial name,status,loc
PPO_CustomEnv_cf5ce_00000,RUNNING,127.0.0.1:18720


2022-02-24 13:25:50,694	WARNING tune.py:593 -- SIGINT received (e.g. via Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C one more time (or send SIGINT/SIGKILL/SIGTERM) to skip. 


Trial name,status,loc
PPO_CustomEnv_cf5ce_00000,RUNNING,127.0.0.1:18720


Trial name,status,loc
PPO_CustomEnv_cf5ce_00000,RUNNING,127.0.0.1:18720


(PPOTrainer pid=18720) 2022-02-24 13:25:50,713	ERROR worker.py:432 -- SystemExit was raised from the worker.
(PPOTrainer pid=18720) Traceback (most recent call last):
(PPOTrainer pid=18720)   File "python/ray/_raylet.pyx", line 770, in ray._raylet.task_execution_handler
(PPOTrainer pid=18720)   File "python/ray/_raylet.pyx", line 591, in ray._raylet.execute_task
(PPOTrainer pid=18720)   File "python/ray/_raylet.pyx", line 629, in ray._raylet.execute_task
(PPOTrainer pid=18720)   File "python/ray/_raylet.pyx", line 636, in ray._raylet.execute_task
(PPOTrainer pid=18720)   File "python/ray/_raylet.pyx", line 640, in ray._raylet.execute_task
(PPOTrainer pid=18720)   File "python/ray/_raylet.pyx", line 589, in ray._raylet.execute_task.function_executor
(PPOTrainer pid=18720)   File "/Users/bumjin/opt/anaconda3/envs/torch/lib/python3.6/site-packages/ray/_private/function_manager.py", line 639, in actor_method_executor
(PPOTrainer pid=18720)     return method(__ray_actor, *args, **kwargs)
(P